In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import presence_of_element_located
from selenium.common.exceptions import NoSuchElementException
import time
import string
import openpyxl
import os
import json

In [2]:
PATH = ("C:\\Users\\fatih\\Downloads\\chromedriver_win32\\chromedriver.exe")
driver = webdriver.Chrome(PATH)

<ipython-input-2-b6bf87c95fd0>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


In [3]:
UoT= 'https://www.google.ca/maps/place/University+of+Toronto+-+St.+George+Campus/@43.6628917,-79.3978504,17z/data=!4m7!3m6!1s0x882b34b8f331fd9b:0x8d1d9bb6765a76f7!8m2!3d43.6628917!4d-79.3956564!9m1!1b1'
ReU= 'https://www.google.com/maps/place/Ryerson+University/@43.6576585,-79.3788017,17z/data=!4m7!3m6!1s0x89d4cb35431c1395:0xe8ed8bd69125d6f4!8m2!3d43.6576585!4d-79.3788017!9m1!1b1'
WeU= 'https://www.google.ca/maps/place/Western+University/@43.0095971,-81.2759223,17z/data=!4m7!3m6!1s0x882eee0e6ac42bd1:0xaa87f353aab9799b!8m2!3d43.0095971!4d-81.2737336!9m1!1b1'
UoO= 'https://www.google.ca/maps/place/University+of+Ottawa/@45.4231064,-75.6853216,17z/data=!4m7!3m6!1s0x4cce050a6db98d73:0x188a59c3622fdbae!8m2!3d45.4231064!4d-75.6831329!9m1!1b1'
YoU= 'https://www.google.ca/maps/place/York+University/@43.7734535,-79.5040571,17z/data=!4m7!3m6!1s0x89d4cd330b767bfb:0xdbb899cc9da76d19!8m2!3d43.7734535!4d-79.5018684!9m1!1b1'


In [4]:
#Opening Google maps url at a time & add sleeping duration to allow for full loading  
driver.get(YoU)
time.sleep(3)

In [5]:
#Creating a scrollable dynamic function to show all the reviews 
scrollable_div_YoU = driver.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[2]')
for i in range(0,10):
    driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scrollable_div_YoU)
    time.sleep(1)

<ipython-input-5-d15cd444648a>:2: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  scrollable_div_YoU = driver.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[2]')


In [6]:
import numpy as np
from Crypto.PublicKey import RSA
import json
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup as bs
import os 
import requests
import pandas as pd
from pprint import pprint
import matplotlib.pyplot as plt
import seaborn as sns
from pymongo import MongoClient
%matplotlib inline

In [7]:
#parsing the webpage
response_YoU = bs(driver.page_source, 'html.parser')
#closing the dirver after parsing it to start building and analysing the outcome
driver.close()

In [8]:
reviews_YoU = response_YoU.find_all('div', class_='ODSEW-ShBeI NIyLF-haAclf gm2-body-2')
len(reviews_YoU)

90

In [9]:
# creating a review summary
# Select the items that will be used in the analysis
def get_review_summary(result_set):
    rev_dict = {'Review Rate': [], 'Review Time': [], 'Review Text': [], 'User': [], 'Impact': []}
    for result in result_set:
        review_rate = result.find('span', class_='ODSEW-ShBeI-H1e3jb')["aria-label"]
        review_time = result.find('span', class_='ODSEW-ShBeI-RgZmSc-date').text
        review_text = result.find('span', class_='ODSEW-ShBeI-text').text
        reviewer = result.find('div', class_='ODSEW-ShBeI-title').text
        Usefulness = result.find('div', class_='ODSEW-ShBeI-VdSJob').text
        rev_dict['Review Rate'].append(review_rate)
        rev_dict['Review Time'].append(review_time)
        rev_dict['Review Text'].append(review_text)
        rev_dict['User'].append(reviewer)
        rev_dict['Impact'].append(Usefulness)

    return(pd.DataFrame(rev_dict))

rev_summ = get_review_summary(reviews_YoU)
rev_summ

,Review Rate,Review Time,Review Text,User,Impact
0,5 stars,a week ago,Great hockey people care and everyone wears a ...,Lil Stochl,Local Guide · 180 reviews
1,5 stars,2 months ago,A great place to learn and meet new friends as...,Calwin,Local Guide · 65 reviews
2,5 stars,3 months ago,York University (French: Université York) is a...,الثريا الثريا,Local Guide · 206 reviews
3,5 stars,2 months ago,I got too excited to visit the campus. The cam...,Lana Jadenson,Local Guide8 reviews
4,3 stars,2 months ago,It is a large campus and very walkable. Unfort...,Hearing Eyes,Local Guide · 28 reviews
...,...,...,...,...,...
85,5 stars,2 years ago,Studied 2 degrees over a 6 year period here be...,Charles Moffat,Local Guide114 reviews
86,5 stars,a year ago,"Huge campus, better bring your running shoes i...",Ian Simpson,Local Guide · 11 reviews
87,5 stars,2 years ago,"As a Chinese student, I would say it is the be...",Alex Le,Local Guide · 210 reviews
88,5 stars,4 years ago,Nice university to walk around!,Zaid Asfour,Local Guide · 157 reviews


# data cleaning

In [10]:
rev_summ['Impact'] = rev_summ['Impact'].map(lambda x: x.lstrip('Local Guide ·'))

In [11]:
rev_summ

,Review Rate,Review Time,Review Text,User,Impact
0,5 stars,a week ago,Great hockey people care and everyone wears a ...,Lil Stochl,180 reviews
1,5 stars,2 months ago,A great place to learn and meet new friends as...,Calwin,65 reviews
2,5 stars,3 months ago,York University (French: Université York) is a...,الثريا الثريا,206 reviews
3,5 stars,2 months ago,I got too excited to visit the campus. The cam...,Lana Jadenson,8 reviews
4,3 stars,2 months ago,It is a large campus and very walkable. Unfort...,Hearing Eyes,28 reviews
...,...,...,...,...,...
85,5 stars,2 years ago,Studied 2 degrees over a 6 year period here be...,Charles Moffat,114 reviews
86,5 stars,a year ago,"Huge campus, better bring your running shoes i...",Ian Simpson,11 reviews
87,5 stars,2 years ago,"As a Chinese student, I would say it is the be...",Alex Le,210 reviews
88,5 stars,4 years ago,Nice university to walk around!,Zaid Asfour,157 reviews


In [12]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\fatih\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [14]:
stop = stopwords.words('english')
rev_summ['Review Text'] = rev_summ['Review Text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
rev_summ

,Review Rate,Review Time,Review Text,User,Impact
0,5 stars,a week ago,Great hockey people care everyone wears mask g...,Lil Stochl,180 reviews
1,5 stars,2 months ago,A great place learn meet new friends students....,Calwin,65 reviews
2,5 stars,3 months ago,York University (French: Université York) publ...,الثريا الثريا,206 reviews
3,5 stars,2 months ago,I got excited visit campus. The campus good ch...,Lana Jadenson,8 reviews
4,3 stars,2 months ago,"It large campus walkable. Unfortunately, nothi...",Hearing Eyes,28 reviews
...,...,...,...,...,...
85,5 stars,2 years ago,Studied 2 degrees 6 year period 1999 2005. I l...,Charles Moffat,114 reviews
86,5 stars,a year ago,"Huge campus, better bring running shoes visiting.",Ian Simpson,11 reviews
87,5 stars,2 years ago,"As Chinese student, I would say best universit...",Alex Le,210 reviews
88,5 stars,4 years ago,Nice university walk around!,Zaid Asfour,157 reviews


In [15]:
description_full = rev_summ.set_index('Review Text')
description_full.head(5)

,Review Rate,Review Time,User,Impact
Review Text,,,,
Great hockey people care everyone wears mask glad hockey back.,5 stars,a week ago,Lil Stochl,180 reviews
A great place learn meet new friends students. Campus large filled amenities - also fun explore.,5 stars,2 months ago,Calwin,65 reviews
"York University (French: Université York) public research university Toronto, Ontario, Canada. It Canada's third-largest university,and approximately 55,700 students, 7,000 faculty staff, 325,000 alumni worldwide.",5 stars,3 months ago,الثريا الثريا,206 reviews
I got excited visit campus. The campus good choices food places. I’m international student Toronto far I’m enjoying view. I’ll coming coming September seems like school clean organized . I cannot wait see inside campus.,5 stars,2 months ago,Lana Jadenson,8 reviews
"It large campus walkable. Unfortunately, nothing see views bland. Everything concentrated food hubs, though libraries university station plus commuting.",3 stars,2 months ago,Hearing Eyes,28 reviews


In [16]:
rev_summ['Review Text'] = rev_summ['Review Text'].str.replace('[^\w\s]',' ')
rev_summ

<ipython-input-16-3283235791aa>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  rev_summ['Review Text'] = rev_summ['Review Text'].str.replace('[^\w\s]',' ')


,Review Rate,Review Time,Review Text,User,Impact
0,5 stars,a week ago,Great hockey people care everyone wears mask g...,Lil Stochl,180 reviews
1,5 stars,2 months ago,A great place learn meet new friends students ...,Calwin,65 reviews
2,5 stars,3 months ago,York University French Université York publ...,الثريا الثريا,206 reviews
3,5 stars,2 months ago,I got excited visit campus The campus good ch...,Lana Jadenson,8 reviews
4,3 stars,2 months ago,It large campus walkable Unfortunately nothi...,Hearing Eyes,28 reviews
...,...,...,...,...,...
85,5 stars,2 years ago,Studied 2 degrees 6 year period 1999 2005 I l...,Charles Moffat,114 reviews
86,5 stars,a year ago,Huge campus better bring running shoes visiting,Ian Simpson,11 reviews
87,5 stars,2 years ago,As Chinese student I would say best universit...,Alex Le,210 reviews
88,5 stars,4 years ago,Nice university walk around,Zaid Asfour,157 reviews


In [17]:
rev_summ.to_csv('newyou.csv')
